# Using _egobox_ optimizer _Egor_

## Installation

In [1]:
%pip install egobox

Note: you may need to restart the kernel to use updated packages.


We import _egobox_ as _egx_ for short.

In [2]:
import numpy as np
import egobox as egx

You may setup the logging level to get optimization progress during the execution

In [3]:
# To display optimization information (none by default)
# import logging
# logging.basicConfig(level=logging.INFO)

## Example 1 : Continuous optimization

### Test functions

In [4]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [5]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


### Continuous optimization

In [6]:
egor = egx.Egor(xspecs_xsinx, n_cstr=n_cstr_xsinx)   # see help(egor) for options

In [7]:
egor = egx.Egor(xspecs_g24, 
                n_doe=10, 
                n_cstr=n_cstr_g24, 
                cstr_tol=[1e-3, 1e-3],
                infill_strategy=egx.InfillStrategy.WB2,
                target=-5.5,
                trego=True,
                # outdir="./out",
                # hot_start=True
            )   

# Specify regression and/or correlation models used to build the surrogates of objective and constraints
#egor = egx.Egor(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [8]:
res = egor.minimize(g24, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

Optimization f=[-5.50793692e+00 -1.35143852e-04 -5.26158286e-05] at [2.32952661 3.17841031]
Optimization history: 
Inputs = [[1.55335241 0.36609899]
 [0.48433871 1.59588025]
 [2.93418005 2.74753497]
 [2.58045769 1.77813431]
 [1.06805821 2.89510289]
 [1.48471388 3.81690289]
 [2.37071614 1.10296966]
 [0.82720969 0.44451136]
 [2.08425444 3.41673721]
 [0.08617534 2.39265705]
 [2.24213008 3.99999996]
 [2.47071614 1.20296966]
 [2.58182726 1.31408077]
 [2.63287837 1.43753756]
 [2.32952661 3.17841031]]
Outputs = [[-1.91945140e+00 -2.59662097e+00 -2.19714000e+00]
 [-2.08021896e+00 -1.48190609e+00 -5.13533364e+00]
 [-5.68171502e+00 -1.42792021e+01  2.68270603e+00]
 [-4.35859200e+00 -4.70895411e+00  1.94930315e-02]
 [-3.96316111e+00 -1.08641233e+00  2.82595019e+00]
 [-5.30161677e+00  6.46292378e-01  1.65905814e+00]
 [-3.47368580e+00 -2.44182980e+00 -1.87313519e+00]
 [-1.27172105e+00 -3.43784549e+00 -1.19300768e-01]
 [-5.50099165e+00  1.35506108e+00 -5.26673877e-01]
 [-2.47883239e+00  3.38256872e-

### Egor as a service: ask-and-tell interface

When the user needs to be in control of the optimization loop, `Egor` can be used as a service. 

For instance with the `xsinx` objective function, we can do:

In [9]:
xlimits = egx.to_specs([[0.0, 25.0]])
egor = egx.Egor(xlimits, seed=42) 

# initial doe
x_doe = egx.lhs(xlimits, 3, seed=42)
y_doe = xsinx(x_doe)
for _ in range(10): # run for 10 iterations
    x = egor.suggest(x_doe, y_doe)  # ask for best location
    x_doe = np.concatenate((x_doe, x))
    y_doe = np.concatenate((y_doe, xsinx(x))) 
res = egor.get_result(x_doe, y_doe)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

Optimization f=[-15.12510279] at [18.9344756]


## Example 2 : Mixed-integer optimization

### Test function

In [10]:
xspecs_mixint_xsinx = [egx.XSpec(egx.XType.INT, [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

### Mixed-integer optimization with _Egor_

In [11]:
egor = egx.Egor(xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(mixint_xsinx, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

Optimization f=[-15.12161154] at [19.]
Optimization history: 
Inputs = [[22.]
 [ 1.]
 [ 9.]
 [25.]
 [21.]
 [20.]
 [19.]]
Outputs = [[ -7.10960014]
 [  1.78601478]
 [  5.41123083]
 [ 11.42919546]
 [-11.44370682]
 [-14.15453288]
 [-15.12161154]]


## Example 3 : More mixed-integer optimization

In the following example we see we can have other special integer type cases, where a component of x can take one value out of a list of ordered values (ORD type) or being like an enum value (ENUM type). Those types differ by the processing related to the continuous relaxation made behind the scene:
* For INT type, resulting float is rounded to the closest int value,
* For ORD type, resulting float is cast to closest value among the given valid ones,
* For ENUM type, one hot encoding is performed to give the resulting value.  

### Test function

In [12]:
# Objective function which takes [FLOAT, ENUM1, ENUM2, ORD] as input
# Note that ENUM values are passed as indice value eg either 0, 1 or 2 for a 3-sized enum  
def mixobj(X):
    # float
    x1 = X[:, 0]
    #  ENUM 1
    c1 = X[:, 1]
    x2 = c1 == 0
    x3 = c1 == 1
    x4 = c1 == 2
    #  ENUM 2
    c2 = X[:, 2]
    x5 = c2 == 0
    x6 = c2 == 1
    # int
    i = X[:, 3]

    y = (x2 + 2 * x3 + 3 * x4) * x5 * x1 + (x2 + 2 * x3 + 3 * x4) * x6 * 0.95 * x1 + i
    return y.reshape(-1, 1)

### Mixed-integer optimization with _Egor_

In [13]:
xtypes = [
    egx.XSpec(egx.XType.FLOAT, [-5.0, 5.0]),
    egx.XSpec(egx.XType.ENUM, tags=["blue", "red", "green"]),
    egx.XSpec(egx.XType.ENUM, xlimits=[2]),
    egx.XSpec(egx.XType.ORD, [0, 2, 3]),
]
egor = egx.Egor(xtypes, seed=42)
res = egor.minimize(mixobj, max_iters=10)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

Optimization f=[-14.25] at [-5.  2.  1.  0.]
Optimization history: 
Inputs = [[ 0.69939824  0.          0.          0.        ]
 [ 4.84411847  1.          0.          0.        ]
 [-4.75038813  1.          0.          2.        ]
 [-1.81967258  2.          1.          2.        ]
 [ 2.46052467  0.          0.          2.        ]
 [-2.82859054  0.          0.          2.        ]
 [ 2.5012666   2.          1.          0.        ]
 [-0.6935668   2.          1.          3.        ]
 [-4.77212695  2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]]
Outp

Note that `x_opt` result contains indices for corresponding optional tags list hence the second component should be read as 0="red", 1="green", 2="blue", while the third component was unamed 0 correspond to first enum value and 1 to the second one.

## Usage

In [14]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(xspecs, n_cstr=0, cstr_tol=None, n_start=20, n_doe=0, doe=None, regr_spec=Ellipsis, corr_spec=Ellipsis, infill_strategy=Ellipsis, q_points=1, par_infill_strategy=Ellipsis, infill_optimizer=Ellipsis, kpls_dim=None, trego=False, n_clusters=1, n_optmod=1, target=Ellipsis, outdir=None, warm_start=False, seed=None)
 |  
 |  Optimizer constructor
 |  
 |  fun: array[n, nx]) -> array[n, ny]
 |       the function to be minimized
 |       fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |          obj is the objective function [n, nx] -> [n, 1]
 |          cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |          an k the number of constraints (n_cstr)
 |          hence ny = 1 (obj) + k (cstrs)
 |       cstr functions are expected be negative (<=0) at the optimum.
 |  
 |   n_cstr (int):
 |       the number of constraint functions.
 |  
 |   cstr_tol (list(n_cstr,)):
 |       List of tolerances for constraints to b